## 머신러닝
- 기계가 학습을 하는 기술을 개발하는 분야
- 데이터 안에서 규칙을 발견하고 그 규칙을 새로운 데이터에 적용해서 새로운 결과를 도출 => 더욱 정교한 미래 예측이 가능해짐


## 딥러닝
- 많은 수의 신경층을 쌓아 입력된 데이터가 여러 단계의 특징 추룰 과정을 거쳐 자동으로 고수준의 추상적인 지식을 추출하는 방식
- 특징 추출과 특징 분류를 학습의 문제로 통합(보다 자동화된 기술)
- 딥러닝을 통해서 신경망에 대한 관심이 다시 늘어남에 따라 머신러닝 연구에 관심을 다시 갖기 시작함


#### Scikit-learn
<img src='./img/sklearn_sheet.PNG'>

## 예측모델 성능평가
***
### o Python에서 데이터셋의 분리방법 관련 모듈 
    - train_test_split : train ,test 으로 분리
    - stratifiedKFold : cross_validation을 사용하기 위한 모듈 Kfold 모듈과 다르게 각 fold 내 데이터의 클래스를 일정하게 유지
    - Cross_val_score : cross valdiation 결과의 정확도를 측정하기 위한 모듈
    

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

## 모델 성능평가 척도
***
***o Confusion matrix*** : 모델의 성능을 통계적인 수치로 시각화하는 방법  
***o Accuracy***  : 모델이 정확하게 분류 또는 예측하는 데이터의 비율  **==** $\frac{TP + TN}{TP + TN + FP + FN}$  
***o Precision*** : 모델이 예측한 데이터 중 검출된 Positive 데이터의 비율 **==** $\frac{TP}{TP+FP}$  (모델의 관점)  
***o Recall*** : 실제 데이터 중 모델이 검출한 Positive 데이터의 비율  **==** $\frac{TP}{TP+FN}$  , ***민감도(sensitive)***  (실제현실의 관점)    
***o F-Measure***  : Precision과 recall을 통합해 나타내는 정확도  **==** $\frac{2TP}{2TP+FN+FP}$  
***o ROC Curve***  : 민감도와 특이도의 관계를 파악할 수 있는 그래프  
***o RMSE*** : 모델 예측값의 오차 계산***

In [2]:
# classificationclassification_report : recall, precision, f-meausre을 측정
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, mean_squared_error


### 모델 성능평가 실습
##### 학습/테스트 데이터 셋 분리 - train_test_split()
- 데이터셋을 분리하지 않고 학습시 예특 정확도가 1.0인 과적합이 발생한다.

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [26]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data =iris.data
train_label = iris.target
dt_clf.fit(train_data, train_label)

# 학습 데이터셋 예측 수행

pred = dt_clf.predict(train_data)
print('예측 정확도 :',accuracy_score(train_label, pred))

예측 정확도 : 1.0


- train_test_split을 통한 데이터 분리를 통한 모델 적합 및 평가

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
dt_clf2 =DecisionTreeClassifier()
X_train , X_test, y_train, y_test =train_test_split(train_data,train_label, test_size= 0.3 , random_state =121)

dt_clf2.fit(X_train, y_train)
pred = dt_clf2.predict(X_test)
print('예측 정확도 {0: .3f}'.format(accuracy_score(y_test,pred)))

예측 정확도  0.956


In [8]:
from sklearn.model_selection import KFold
import numpy as np

- K Fold CV ( Cross Validation)  
        o .split() 폴드별 학습용, 검증용 테스트의 로우 인덱스를 array 반환

In [28]:
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=  156)

kfold =KFold(n_splits=5)
cv_accuracy = [] # Cross Validation 폴드별 결과를 누적
print('붓꽃 데이터 세트 크기 :', features.shape[0])

붓꽃 데이터 세트 크기 : 150


In [14]:
n_iter =0 
# Kfold는 성능 평가를 보기위해 해보는 것 -> 그대로 모델을 적용하겠다 X
for train_index , test_index, in kfold.split(features):
    # Kfold.split()으로 반환된 인덱스를 이용하여 학습용 , 검증용 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index] , label[test_index]
    #학습 및 예측
    dt_clf.fit(X_train, y_train) # .fit 을 사용시 누적이 되지 않는다.
    pred = dt_clf.predict(X_test)
    n_iter += 1
    # 반복 시 마다 정확도 측정
    accuracy = np.round(accuracy_score(y_test ,pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    #개별 iteration별 정확도를 합하여 평균 정확도 계산
    print('\n#{} 교차 검증 정확도 : {} , 학습 데이터의 크기 : {}, 검증 데이터 크기 : {} '.format(n_iter, accuracy , train_size , test_size))
    print('#{} 검증 세트 인덱스 : {} '.format(n_iter , test_index))
    cv_accuracy.append(accuracy)

print('\n ## 평균 검증 정확도 : ', np.mean(cv_accuracy))


#1 교차 검증 정확도 : 1.0 , 학습 데이터의 크기 : 120, 검증 데이터 크기 : 30 
#1 검증 세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29] 

#2 교차 검증 정확도 : 0.9667 , 학습 데이터의 크기 : 120, 검증 데이터 크기 : 30 
#2 검증 세트 인덱스 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59] 

#3 교차 검증 정확도 : 0.8667 , 학습 데이터의 크기 : 120, 검증 데이터 크기 : 30 
#3 검증 세트 인덱스 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89] 

#4 교차 검증 정확도 : 0.9333 , 학습 데이터의 크기 : 120, 검증 데이터 크기 : 30 
#4 검증 세트 인덱스 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119] 

#5 교차 검증 정확도 : 0.7333 , 학습 데이터의 크기 : 120, 검증 데이터 크기 : 30 
#5 검증 세트 인덱스 : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149] 

 ## 평균 검증 정확도 :  0.9


#### Pandas를 이용한 테이블 

In [17]:
import pandas as pd

iris_df = pd.DataFrame(data= features , columns = iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [21]:
kfold = KFold(n_splits= 3)

n_iter =0
for train_index, test_index in kfold.split(iris_df):
    n_iter +=1
    label_train = iris_df['label'].loc[train_index]
    label_test = iris_df['label'].loc[test_index]
    
    print('## 교차 검증 {}'.format(n_iter))
    print('학습 레이블 데이터 분포 : \n' ,label_train.value_counts())
    print('검증 레이블 데이터 분포 : \n' ,label_test.value_counts())

## 교차 검증 1
학습 레이블 데이터 분포 : 
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 0    50
Name: label, dtype: int64
## 교차 검증 2
학습 레이블 데이터 분포 : 
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 1    50
Name: label, dtype: int64
## 교차 검증 3
학습 레이블 데이터 분포 : 
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    50
Name: label, dtype: int64


#### StratifiedKFold를 이용한 kFold 교차검증

In [22]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter= 0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter +=1
    label_train = iris_df['label'].loc[train_index]
    label_test = iris_df['label'].loc[test_index]
    print('## 교차 검증 : {} '.format(n_iter))
    print('학습 레이블 데이터 분포 : \n' , label_train.value_counts())
    print('검증 레이블 데이터 분포 : \n' , label_test.value_counts())

## 교차 검증 : 1 
학습 레이블 데이터 분포 : 
 2    34
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 1    17
0    17
2    16
Name: label, dtype: int64
## 교차 검증 : 2 
학습 레이블 데이터 분포 : 
 1    34
2    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    17
0    17
1    16
Name: label, dtype: int64
## 교차 검증 : 3 
학습 레이블 데이터 분포 : 
 0    34
2    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포 : 
 2    17
1    17
0    16
Name: label, dtype: int64


In [30]:
dt_clf = DecisionTreeClassifier(random_state=156)

skfold  = StratifiedKFold(n_splits= 3)
n_iter =0
cv_accuracy =[]

for train_index, test_index in skfold.split(features, label):
    X_train, X_test = features[train_index] , features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    n_iter +=1
    accuracy = np.round(accuracy_score(y_test,pred),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print('##{} 교차 검증 정확도 : {}, 학습 데이터 크기 {}, 검증 데이터 크기 : {}'.format(n_iter, accuracy, train_size, test_size))
    print('#{} 검증 세트 인덱스 : {} '.format(n_iter, test_index))
    
    cv_accuracy.append(accuracy)
    
print('\n ## 교차 검증 정확도 : ', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도 : %.3f'%np.mean(cv_accuracy))

##1 교차 검증 정확도 : 0.98, 학습 데이터 크기 100, 검증 데이터 크기 : 50
#1 검증 세트 인덱스 : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115] 
##2 교차 검증 정확도 : 0.94, 학습 데이터 크기 100, 검증 데이터 크기 : 50
#2 검증 세트 인덱스 : [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132] 
##3 교차 검증 정확도 : 0.98, 학습 데이터 크기 100, 검증 데이터 크기 : 50
#3 검증 세트 인덱스 : [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149] 

 ## 교차 검증 정확도 :  [0.98 0.94 0.98]
## 평균 검증 정확도 : 0.967


- cross_val_score()
        o 앞서 반복문을 사용한 교차 검증을 cross_validate함수를 통해 가능 

In [33]:
from sklearn.model_selection import cross_validate

scores = cross_val_score(dt_clf, features, label, scoring = 'accuracy', cv=3)
print(' 교차 검증별 정확도 :' , np.round(scores,4))
print(' 평균 검증 정확도 : ', np.round(np.mean(scores),4))

 교차 검증별 정확도 : [0.98 0.94 0.98]
 평균 검증 정확도 :  0.9667


#### GridSearchCV 
    -param_grid : param의 사전 타입
    - refit : Refit an estimator using the best found parameters on the whole dataset.

In [50]:
from sklearn.model_selection import GridSearchCV

para_meter = {'max_depth': [1,2,3] , 'min_samples_split':[i for i in range(10,20)]} # 조합해보고 싶은 parameter를 모두 넣는다. 단, Dictionary 타입으로

grid_dtree = GridSearchCV(dt_clf, param_grid=para_meter , cv= 3, refit =True)
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data , iris.target , test_size=0.2, random_state =121)

grid_dtree.fit(X_train, y_train)

score_df = pd.DataFrame(grid_dtree.cv_results_)
score_df[['params', 'mean_test_score','rank_test_score','split0_test_score', 'split1_test_score','split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 10}",0.700000,21,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 11}",0.700000,21,0.700,0.7,0.70
2,"{'max_depth': 1, 'min_samples_split': 12}",0.700000,21,0.700,0.7,0.70
3,"{'max_depth': 1, 'min_samples_split': 13}",0.700000,21,0.700,0.7,0.70
4,"{'max_depth': 1, 'min_samples_split': 14}",0.700000,21,0.700,0.7,0.70
5,"{'max_depth': 1, 'min_samples_split': 15}",0.700000,21,0.700,0.7,0.70
6,"{'max_depth': 1, 'min_samples_split': 16}",0.700000,21,0.700,0.7,0.70
7,"{'max_depth': 1, 'min_samples_split': 17}",0.700000,21,0.700,0.7,0.70
8,"{'max_depth': 1, 'min_samples_split': 18}",0.700000,21,0.700,0.7,0.70
9,"{'max_depth': 1, 'min_samples_split': 19}",0.700000,21,0.700,0.7,0.70


In [43]:
grid_dtree.best_params_

{'max_depth': 3, 'min_samples_split': 2}

- refit으로 인해 최고 성능의 모델이 반환된 상태
        o model.best_estimator_

In [49]:
estimator = grid_dtree.best_estimator_

pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도 : {0:.4f} '.format(accuracy_score(pred,y_test)))

테스트 데이터 세트 정확도 : 0.9667 


- DecisionTreeClassifier() 파라미터
        o mat_depth : 과적합을 방지하기 위한 최대 깊이를 규제하기 위한 파라미터
        o min_samples_split : 각 분할에 대한 최소 표본 수